## Cell1：导入所需的库文件

In [95]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import cv2
import pickle
import random
from typing import List, Dict, Any,Tuple
from scipy.spatial.transform import Rotation as R_scipy
import open3d as o3d
from contextlib import redirect_stdout
from feature_extraction import load_or_extract_features
from feature_matching import load_or_match_image_pairs, filter_two_view_geometry
from initial_recon import print_pose_info, pick_initial_pair
from pnp_recon import pnp_pose, visualize_with_open3d, triangulate_points,multi_view_triangulation,visualize_colored_point_cloud,register_frame_by_epipolar
from bundle_adjustment import BundleAdjustment,TorchBundleAdjustment
from metrics import reprojection_errors,reprojection_stats,track_length_stats,image_coverage_stats,points_per_camera_ratio


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 指定所需要的变量路径

In [78]:
# User parameters: adjust these paths before running
IMAGE_FOLDER = r"D:\lyz\Python\CV\AI4701\final project\images"
INTRINSICS_FILE = r"D:\lyz\Python\CV\AI4701\final project\camera_intrinsic.txt"
OUTPUT_DIR = r"D:\lyz\Python\CV\AI4701\final project\ablation_output"
SAVE_FEATURES = False
LOAD_FEATURES = False
VERBOSE = True


## Cell 2: Load Images and Intrinsics

In [79]:

def load_images(image_folder: str) -> List[str]:
    paths = []
    for root, dirs, files in os.walk(image_folder):
        # 1. 只保留图片文件
        image_files = [
            f for f in files
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        # 2. 按文件名逆序排序（"163225" > "163224"）
        image_files.sort(reverse=True)
        # 3. 拼接完整路径
        for f in image_files:
            paths.append(os.path.join(root, f))
    return paths


def load_intrinsics(k_path: str) -> np.ndarray:
    if not os.path.exists(k_path):
        raise FileNotFoundError(f"相机内参文件未找到：{k_path}")
    K = np.loadtxt(k_path, dtype=np.float32)
    if K.shape != (3, 3):
        raise ValueError(f"相机内参形状错误，应为3x3，但得到: {K.shape}")
    return K


In [80]:
# Load at module level
image_paths = load_images(IMAGE_FOLDER)
image_paths = image_paths[:10]  # Limit to first 100 images for testing
K = load_intrinsics(INTRINSICS_FILE)

## Cell 3: Feature Extraction and Matching

In [85]:
def run_feature_pipeline(paths: List[str], K: np.ndarray, output_dir: str,
                         save: bool, load: bool, verbose: bool=True):
    def _run_quiet(func, *args, **kwargs):
        if verbose:
            return func(*args, **kwargs)
        with open(os.devnull, 'w') as devnull:
            with redirect_stdout(devnull):
                return func(*args, **kwargs)

    print("Extracting features...")
    features = _run_quiet(
        load_or_extract_features,
        paths, method='sift_improved', save=save, load=load, output_dir=output_dir
    )

    print("Matching features...")
    raw_matches = _run_quiet(
        load_or_match_image_pairs,
        features, method='sift', save=save, load=load, output_dir=output_dir
    )

    # RANSAC filtering
    n = len(features)
    al_matches = [[[] for _ in range(n)] for __ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j: continue
            kp1, _ = features[i]
            kp2, _ = features[j]
            m = raw_matches[i][j]
            if len(m) < 4:
                continue
            inliers, _ = _run_quiet(filter_two_view_geometry, kp1, kp2, m, K)
            # # 1) 从 m（List[cv2.DMatch]）里取坐标
            # pts1 = np.float32([ kp1[mi.queryIdx].pt for mi in m ])  # (N,2)
            # pts2 = np.float32([ kp2[mi.trainIdx].pt for mi in m ])  # (N,2)

            # # 2) 用 Essential 矩阵 + RANSAC 得到 mask
            # #    参数 threshold 可以根据你的像素精度调，比如 1.0–2.0 像素
            # E, mask = cv2.findEssentialMat(
            #     pts1, pts2, 
            #     cameraMatrix=K,
            #     method=cv2.RANSAC,
            #     prob=0.999,
            #     threshold=1.0
            # )
            # # mask 是一个 N×1 的 ndarray，值为 1 表示 inlier，0 表示 outlier

            # # 3) 根据 mask 筛出 inliers matches
            # inliers = [ m[i] for i in range(len(m)) if mask[i,0] == 1 ]

            # 4) 存回你的 al_matches
            al_matches[i][j] = inliers

    # Flatten into pairs
    matches_list = []
    for i in range(n):
        for j in range(i+1, n):
            if len(al_matches[i][j]) >= 4:
                matches_list.append((i, j, al_matches[i][j]))

    return features, al_matches, matches_list

# Execute
features, al_matches, matches_list = run_feature_pipeline(
    image_paths, K, OUTPUT_DIR, SAVE_FEATURES, LOAD_FEATURES, VERBOSE
)



Extracting features...
开始提取特征…
=== 特征提取完成: 10/10 张图像有效 ===
Matching features...
[DEBUG] 未加载缓存，开始重新匹配（不保存）…

=== [SIFT] 两两特征匹配开始 ===
Matching features: 4858 vs 4490 descriptors
Found 1468 good matches out of 4858 total matches
Match ratio: 30.2%
Distance stats - Min: 0.05, Max: 0.41, Mean: 0.19
Matching features: 4858 vs 4199 descriptors
Found 1012 good matches out of 4858 total matches
Match ratio: 20.8%
Distance stats - Min: 0.08, Max: 0.39, Mean: 0.21
Matching features: 4858 vs 4486 descriptors
Found 572 good matches out of 4858 total matches
Match ratio: 11.8%
Distance stats - Min: 0.11, Max: 0.38, Mean: 0.23
Matching features: 4858 vs 4835 descriptors
Found 461 good matches out of 4858 total matches
Match ratio: 9.5%
Distance stats - Min: 0.11, Max: 0.44, Mean: 0.24
Matching features: 4858 vs 4578 descriptors
Found 373 good matches out of 4858 total matches
Match ratio: 7.7%
Distance stats - Min: 0.10, Max: 0.37, Mean: 0.25
Matching features: 4858 vs 4677 descriptors
Found 268 good


## Cell 4: Initial Pose Estimation

In [86]:

def initialize_reconstruction(K: np.ndarray,
                              al_matches: List[List[List[cv2.DMatch]]],
                              features: List[Tuple[List[cv2.KeyPoint], np.ndarray]]):
    print("Estimating initial pose...")
    keypoints = [kp for kp, _ in features]
    i, j, R, t, inliers = pick_initial_pair(K, al_matches, keypoints=keypoints)
    print(f"Initial pair: {i}, {j}")
    print_pose_info(R, t, len(inliers), len(inliers))
    return i, j, R, t, inliers

init_i, init_j, R0, t0, init_inliers = initialize_reconstruction(K, al_matches, features)
print("Initial pair selected:", init_i, init_j)

Estimating initial pose...
Initial pair: 3, 4
SCENE INITIALIZATION RESULTS

1. Feature Matching:
   - Total matches: 746
   - Inlier matches: 746
   - Inlier ratio: 1.000

2. Rotation Matrix (R):
[[ 9.99175463e-01 -1.12709648e-04 -4.06002662e-02]
 [ 1.18079132e-03  9.99653812e-01  2.62842386e-02]
 [ 4.05832484e-02 -2.63105067e-02  9.98829694e-01]]

3. Translation Vector (t):
[ 0.98728219 -0.01739318  0.1580233 ]

4. Rotation as Rodrigues vector: [-0.02630764 -0.0406076   0.000647  ]
5. Rotation in degrees: [-1.50731662 -2.32664424  0.03707054]
6. Translation distance (normalized): 1.0000
7. Total rotation angle: 2.77 degrees
Initial pair selected: 3 4


In [87]:
# %% 
# 7. Triangulate initial 3D points between camera init_i (ID 0) and init_j (ID 1)
kp_i, _ = features[init_i]
kp_j, _ = features[init_j]


pts3d, valid_idxs = triangulate_points(
    init_inliers, 
    kp_i, 
    kp_j, 
    K,
    np.eye(3),                # cam0 是世界坐标
    np.zeros((3,1)),         
    R0, 
    t0
)

print(f"Triangulated {pts3d.shape[0]} 3D points from the initial pair.")


# 8. 构建点云字典和观测字典
#    points_3d: pid -> [x, y, z]
#    point_observations: pid -> {cam_idx: (u,v), ...}
points_3d = {}
point_observations = {}

for pid, match_idx in enumerate(valid_idxs):
    # match_idx 是 init_inliers 中的下标
    m = init_inliers[match_idx]
    # 存坐标
    points_3d[pid] = pts3d[pid]
    # 存图像观测
    pt_i = kp_i[m.queryIdx].pt
    pt_j = kp_j[m.trainIdx].pt
    point_observations[pid] = {
        init_i: pt_i,
        init_j: pt_j
    }

print(f"Built observations for {len(points_3d)} points.")


Triangulated 746 3D points from the initial pair.
Built observations for 746 points.


In [88]:
obs_index = {}
for pid, match_idx in enumerate(valid_idxs):
    m = init_inliers[match_idx]
    obs_index[(init_i, m.queryIdx)] = pid
    obs_index[(init_j, m.trainIdx)] = pid

## Cell 5: Incremental SfM Reconstruction

In [89]:
def generate_bounce_path(
    num_images: int,
    begin: int,
    step: int = 10,
    p_jump: float = 0.01
) -> List[int]:
    """
    在 [0, num_images-1] 上以 step 为步长折返行走，同时
    以 p_jump 的概率做一次随机跳跃，以保证所有索引都有机会被访问。
    
    Args:
        num_images: 总图像数量 N
        begin:      起始索引
        step:       固定步长
        p_jump:     每一步有 p_jump 的概率随机跳到任意一个索引

    Returns:
        path: 访问序列，包含 0…num_images-1 中每个索引一次
    """
    visited = set()
    path = []
    pos = begin
    delta = step

    while len(visited) < num_images:
        # 随机跳跃
        if random.random() < p_jump:
            pos = random.randrange(num_images)
            # 随机换个方向
            delta = step if random.random() < 0.5 else -step
        else:
            # 折返逻辑
            if pos < 0:
                pos = -pos
                delta = -delta
            elif pos > num_images - 1:
                pos = 2*(num_images-1) - pos
                delta = -delta

        # 记录新索引
        if pos not in visited:
            visited.add(pos)
            path.append(pos)

        pos += delta

    return path

In [90]:
# 测试一下
order = generate_bounce_path(len(image_paths),begin=init_i , step=3)
print("访问顺序：", order)
# 例如： [0, 5, 10, 15, 19, 14, 9, 4, 0, 6, 12, 18, 23, ...]（直到覆盖所有）

访问顺序： [3, 6, 9, 0, 7, 4, 1, 2, 5, 8]


In [97]:

anchor_idxs  = [init_i, init_j]
camera_poses = {
    init_i: {'R': np.eye(3), 't': np.zeros((3,1))},
    init_j: {'R': R0,        't': t0}
}
# points_3d, point_observations, obs_index 都已准备好

# 2. 准备可变顺序列表，从 init_i 开始
start = order.index(init_i)
order_list = order[start:]  # e.g. [init_i, next, next, ...]
# 我们用 idx_ptr 来遍历 order_list
idx_ptr = 1
iteration = 1  

# 3. 增量主循环
while idx_ptr < len(order_list):
    prev_idx = order_list[idx_ptr - 1]
    curr_idx = order_list[idx_ptr]

    # 3.1 如果前一帧还没注册，直接删掉这帧
    if prev_idx not in camera_poses:
        if VERBOSE:
            print(f"跳过 {prev_idx}->{curr_idx}：没有已知位姿，移除 {curr_idx}")
        order_list.pop(idx_ptr)
        continue

    # 3.2 聚合所有 anchors 的 2D–3D 对应
    kp_curr, _ = features[curr_idx]
    img_pts, obj_pts, corr = [], [], []
    for anchor in anchor_idxs:
        kp_a, _ = features[anchor]
        for m in al_matches[anchor][curr_idx]:
            key = (anchor, m.queryIdx)
            if key in obs_index:
                pid = obs_index[key]
                img_pts.append(kp_curr[m.trainIdx].pt)
                obj_pts.append(points_3d[pid])
                corr.append(m)
    # 如果对应太少，删掉这帧
    if len(obj_pts) < 4:
        if VERBOSE:
            print(f"跳过 图 {curr_idx}：2D–3D 对应 < 4 ({len(obj_pts)})，移除 {curr_idx}")
        order_list.pop(idx_ptr)
        continue

    # 3.3 PnP 注册
    # Rn, tn, inlier_mask = pnp_pose(img_pts, obj_pts, K)
    Rn, tn, inlier_mask = register_frame_by_epipolar(
        prev_idx, curr_idx,
        al_matches, features, K
    )
    if Rn is None:
        if VERBOSE:
            print(f"跳过 {prev_idx}->{curr_idx}：对极注册失败")
        order_list.pop(idx_ptr)
        continue
    camera_poses[curr_idx] = {'R': Rn, 't': tn}
    if VERBOSE:
        print(f"注册帧 {curr_idx}：PnP 内点 {int(inlier_mask.sum())}/{len(obj_pts)}")

    # 3.4 多视图三角化
    new_pts = multi_view_triangulation(
        new_idx=curr_idx,
        anchor_idxs=anchor_idxs,
        features=features,
        al_matches=al_matches,
        camera_poses=camera_poses,
        points_3d=points_3d,
        K=K
    )

    # 3.5 更新全局点云和观测
    for pid, X, match_map in new_pts:
        points_3d[pid] = X
        obs = {}
        for cam_idx, m in match_map.items():
            kpt = m.trainIdx if cam_idx == curr_idx else m.queryIdx
            uv  = features[cam_idx][0][kpt].pt
            obs[cam_idx] = uv
            obs_index[(cam_idx, kpt)] = pid
        point_observations[pid] = obs
        
    if VERBOSE:
        print(f"  新增 {len(new_pts)} 点，总共 {len(points_3d)} 点")

    # 3.5.5 局部 Bundle Adjustment
    # 选出本地要优化的相机和点
    if iteration % 5 == 0:
        local_cams = anchor_idxs.copy()      # e.g. 最近 5 帧
        # 找到这些相机观测到的所有点 pid
        local_pts = set()
        for pid, obs in point_observations.items():
            if any(c in local_cams for c in obs):
                local_pts.add(pid)
        local_pts = sorted(local_pts)

        # 构造 BA 输入：齐次相机矩阵数组 + 点坐标数组 + obs_uv, cam_idxs, pt_idxs
        # 1) 相机矩阵列表
        cam_mats_local = []
        cam_idx_map    = {}  # 原全局索引 → 本地 BA 内索引
        for i, c in enumerate(local_cams):
            cam_pose = camera_poses[c]
            T = np.eye(4, dtype=np.float64)
            T[:3,:3] = cam_pose['R']
            T[:3, 3] = cam_pose['t'].reshape(3)
            cam_mats_local.append(T)
            cam_idx_map[c] = i
        cam_mats_local = np.stack(cam_mats_local, axis=0)  # (Nc_local,4,4)

        # 2) 点坐标数组 + pid → 本地点索引
        pts_local = np.vstack([points_3d[pid] for pid in local_pts])  # (Np_local,3)
        pt_idx_map = {pid:i for i,pid in enumerate(local_pts)}

        # 3) 观测展平（仅保留本地 cams & pts）
        obs_uv_ba   = []
        cam_idxs_ba = []
        pt_idxs_ba  = []
        for pid in local_pts:
            for c, uv in point_observations[pid].items():
                if c in cam_idx_map:
                    obs_uv_ba  .append(uv)
                    cam_idxs_ba.append(cam_idx_map[c])
                    pt_idxs_ba .append(pt_idx_map[pid])
        obs_uv_ba   = np.array(obs_uv_ba,   dtype=np.float64)
        cam_idxs_ba = np.array(cam_idxs_ba, dtype=np.int64)
        pt_idxs_ba  = np.array(pt_idxs_ba,  dtype=np.int64)

        # 4) 运行本地 BA
        ba = TorchBundleAdjustment(
            camera_intrinsics=K,
            init_cameras=cam_mats_local,
            init_points=pts_local
        )
        opt_cams, opt_pts, info = ba.optimize(
            cameras=cam_mats_local,
            points_3d=pts_local,
            observations=obs_uv_ba,
            camera_indices=cam_idxs_ba,
            point_indices=pt_idxs_ba
        )

        # 5) 把优化结果写回全局
        # 5.1 相机
        for i, c in enumerate(local_cams):
            Topt = opt_cams[i]
            camera_poses[c]['R'] = Topt[:3,:3]
            camera_poses[c]['t'] = Topt[:3,3].reshape(3,1)
        # 5.2 点
        for i, pid in enumerate(local_pts):
            points_3d[pid] = opt_pts[i]

        if VERBOSE:
            print(f"  局部 BA 完成，优化 {len(local_cams)} 台相机和 {len(local_pts)} 个点")

    # 3.6 更新 anchor 队列（RFU：最近 5 帧）
    anchor_idxs.append(curr_idx)
    if len(anchor_idxs) > 5:
        dropped = anchor_idxs.pop(0)
        if VERBOSE:
            print(f"  anchor 满 {5}，丢弃最早帧 {dropped}")

    # 3.7 前进到下一帧
    idx_ptr += 1
    iteration += 1

print("增量重建完成")
print(f"相机数: {len(camera_poses)}, 三维点数: {len(points_3d)}")

注册帧 6：PnP 内点 399/678
  新增 474 点，总共 1220 点
注册帧 9：PnP 内点 448/1014
  新增 28 点，总共 1248 点
注册帧 0：PnP 内点 291/580
  新增 33 点，总共 1281 点
注册帧 7：PnP 内点 178/917
  新增 1 点，总共 1282 点
  anchor 满 5，丢弃最早帧 3
注册帧 4：PnP 内点 562/372
  新增 0 点，总共 1282 点
[rodrigues] input rvecs shape: torch.Size([128, 3])
[rodrigues] output R shape: torch.Size([128, 3, 3])
[forward] batch obs_uv shape: torch.Size([128, 2]), pred_uv shape: torch.Size([128, 2])
[forward] sample obs_uv: [[1601.771  1311.9738]
 [1635.0151 1377.3965]
 [1622.8323 1430.9491]], pred_uv: [[1682.9207 1311.0222]
 [1635.033  1377.5671]
 [1684.7458 1423.3632]]
[rodrigues] input rvecs shape: torch.Size([128, 3])
[rodrigues] output R shape: torch.Size([128, 3, 3])
[forward] batch obs_uv shape: torch.Size([128, 2]), pred_uv shape: torch.Size([128, 2])
[forward] sample obs_uv: [[2162.7876 1188.7339]
 [2242.516  1880.9976]
 [1721.1378 1509.0712]], pred_uv: [[2219.5264 1181.5317]
 [2200.7634 1852.8807]
 [1717.3035 1506.982 ]]
[rodrigues] input rvecs shape: torch.Siz

In [99]:
poses = sorted(camera_poses.keys())
sorted_pids = sorted(points_3d.keys())
points = np.vstack([points_3d[pid].reshape(1,3) for pid in sorted_pids])
print(f"Converted to points array of shape {points.shape}")

# 2. 构造 cams 列表
cams = []
for idx in poses:
    cam = camera_poses[idx]
    T = np.eye(4, dtype=np.float32)
    T[:3, :3] = cam['R']
    T[:3,  3] = cam['t'].reshape(3,)
    cams.append(T)
print(f"Built {len(cams)} camera poses as 4×4 matrices")


Converted to points array of shape (1338, 3)
Built 10 camera poses as 4×4 matrices


## Cell 6: Recentering and Camera Matrices

In [100]:
def recenter_poses_and_points(
    poses: List[Dict[str, Any]],
    points_3d: np.ndarray,
    world_filename: str,
    paths: List[str]
) -> Tuple[List[Dict[str, np.ndarray]], np.ndarray]:
    """
    对相机位姿和三维点做“重心化”处理：
    - world_filename 对应的那台相机 将成为新的原点且朝向不变
    - 其它相机与三维点 都按 R_w*(X - t_w) 变换到新坐标系

    Args:
        poses: List of {'R': (3,3), 't': (3,)}，原始世界坐标系下的相机外参
        points_3d: (N_pts,3) ndarray，原始世界坐标系下的三维点
        world_filename: 要作为新世界原点的那张图的文件名（例如 image_paths[0] 的 basename）
        paths:     List of 对应的所有图片完整路径

    Returns:
        new_poses: List of {'R': (3,3), 't': (3,)}，重心化后的相机外参
        new_points: (N_pts,3) ndarray，重心化后的三维点坐标
    """
    # 找到 world 相机在 poses/paths 中的索引
    idx_world = next(
        i for i, p in enumerate(paths)
        if os.path.basename(p) == world_filename
    )
    R_w = poses[idx_world]['R']
    t_w = poses[idx_world]['t'].reshape(3,)

    # 1) 重心化相机位姿
    new_poses = []
    for pose in poses:
        R_k = pose['R']
        t_k = pose['t'].reshape(3,)

        # 新的旋转、平移
        Rk_new = R_k @ R_w.T
        tk_new = t_k - (Rk_new @ t_w)

        new_poses.append({'R': Rk_new, 't': tk_new})

    # 2) 重心化三维点：X' = R_w @ (X - t_w)
    #    points_3d: (N_pts,3) → (3, N_pts) → 变换 → (N_pts,3)
    shifted = points_3d - t_w[None, :]        # (N_pts,3)
    new_points = (R_w @ shifted.T).T         # (N_pts,3)

    return new_poses, new_points

In [101]:

sorted_pids = sorted(points_3d.keys())
points = np.vstack([points_3d[pid].reshape(1,3) for pid in sorted_pids])
world_filename = "DJI_20200223_163225_243.jpg"
# 2. 把 camera_poses 转成和 image_paths 一一对应的列表
poses_list = []
valid_paths = []
for idx, path in enumerate(image_paths):
    cam = camera_poses.get(idx)
    if cam is None:
        continue
    R = cam['R']
    t = cam['t'].reshape(3,)    # 从 (3,1) → (3,)
    poses_list.append({'R': R, 't': t})
    valid_paths.append(path)

# 3. 调用 recenter_poses_and_points
new_poses_list, new_points = recenter_poses_and_points(
    poses_list,
    points,
    world_filename,
    valid_paths
)

# 4. 把 new_poses_list 再转回和 camera_poses 格式一致的 dict
new_camera_poses = {}
for idx, pose in enumerate(new_poses_list):
    new_camera_poses[idx] = {
        'R': pose['R'],
        't': pose['t'].reshape(3,1)   # 恢复成 (3,1)
    }


cams = []
for idx in sorted(new_camera_poses.keys()):
    cam = new_camera_poses[idx]
    T = np.eye(4, dtype=np.float32)
    T[:3,:3] = cam['R']
    T[:3, 3] = cam['t'].reshape(3,)
    cams.append(T)

# new_poses 是 [{'R':..., 't':...}, …]
# new_points 形状 (N_pts,3)


## Cell 7: Bundle Adjustment

In [102]:
# 1) 按帧索引排序，构造相机齐次矩阵列表
sorted_cam_idxs = sorted(new_camera_poses.keys())
camera_matrices = []
for idx in sorted_cam_idxs:
    cam = new_camera_poses[idx]
    T = np.eye(4, dtype=np.float32)
    T[:3, :3] = cam['R']
    T[:3,  3] = cam['t'].reshape(3,)
    camera_matrices.append(T)
camera_matrices = np.stack(camera_matrices, axis=0)  # (N_cam,4,4)
print("相机矩阵形状:", camera_matrices.shape)

# 2) 把 points_3d 从 dict → (N_pt,3) ndarray
sorted_pt_ids = sorted(points_3d.keys())
ptid_to_idx = {pid:i for i,pid in enumerate(sorted_pt_ids)}
points_array = np.vstack([points_3d[pid].reshape(1,3) for pid in sorted_pt_ids])
points_array = points_array.astype(np.float32)       # (N_pt,3)
print("三维点数组形状:", points_array.shape)

# 3) 从 point_observations 拆出 obs_uv, camera_idxs, point_idxs
obs_uv      = []
camera_idxs = []
point_idxs  = []

for pid, cam_dict in point_observations.items():
    if pid not in ptid_to_idx:
        continue
    pt_idx = ptid_to_idx[pid]
    for cam_idx, uv in cam_dict.items():
        # 跳过那些未注册的相机
        if cam_idx not in new_camera_poses:
            continue
        # 把 cam_idx 转换到相机矩阵列表中的位置
        cam_list_idx = sorted_cam_idxs.index(cam_idx)
        obs_uv.append(uv)
        camera_idxs.append(cam_list_idx)
        point_idxs.append(pt_idx)

obs_uv      = np.array(obs_uv,      dtype=np.float32)  # (N_obs,2)
camera_idxs = np.array(camera_idxs, dtype=np.int64)    # (N_obs,)
point_idxs  = np.array(point_idxs,  dtype=np.int64)    # (N_obs,)
print("观测数量:", obs_uv.shape[0])

相机矩阵形状: (10, 4, 4)
三维点数组形状: (1338, 3)
观测数量: 2676


In [103]:
# 4) 构造并运行 Bundle Adjustment
ba = TorchBundleAdjustment(
    camera_intrinsics=K, 
    init_cameras=camera_matrices,
    init_points=points_array
)
opt_cams, opt_pts, info = ba.optimize(
    cameras=camera_matrices,
    points_3d=points_3d,
    observations=obs_uv,
    point_indices=point_idxs,
    camera_indices=camera_idxs
)



[rodrigues] input rvecs shape: torch.Size([128, 3])
[rodrigues] output R shape: torch.Size([128, 3, 3])
[forward] batch obs_uv shape: torch.Size([128, 2]), pred_uv shape: torch.Size([128, 2])
[forward] sample obs_uv: [[1914.6204  1893.8165 ]
 [1843.9779  1517.1796 ]
 [ 355.81845  511.8084 ]], pred_uv: [[1469.8751  2174.5103 ]
 [1392.319   1786.4644 ]
 [ 264.66968  775.28467]]
[rodrigues] input rvecs shape: torch.Size([128, 3])
[rodrigues] output R shape: torch.Size([128, 3, 3])
[forward] batch obs_uv shape: torch.Size([128, 2]), pred_uv shape: torch.Size([128, 2])
[forward] sample obs_uv: [[1953.6233  1131.9387 ]
 [1417.1683  1011.24756]
 [ 429.3999  1348.5549 ]], pred_uv: [[1474.291  1379.3684]
 [1184.5869 1286.9719]
 [-270.2351 1606.5372]]
[rodrigues] input rvecs shape: torch.Size([128, 3])
[rodrigues] output R shape: torch.Size([128, 3, 3])
[forward] batch obs_uv shape: torch.Size([128, 2]), pred_uv shape: torch.Size([128, 2])
[forward] sample obs_uv: [[2002.4377   1121.3077  ]
 [15


## Cell 8: Save Results

In [104]:

def save_outputs(points: np.ndarray, cameras: List[np.ndarray], output_dir: str):
    os.makedirs(output_dir, exist_ok=True)
    ply = os.path.join(output_dir, 'optimized_point_cloud.ply')
    with open(ply, 'w') as f:
        f.write('ply\nformat ascii 1.0\n')
        f.write(f'element vertex {points.shape[0]}\n')
        f.write('property float x\nproperty float y\nproperty float z\nend_header\n')
        for p in points:
            f.write(f"{p[0]} {p[1]} {p[2]}\n")
    traj = os.path.join(output_dir, 'optimized_camera_trajectory.txt')
    with open(traj, 'w') as f:
        f.write('# CamIdx tx ty tz qx qy qz qw\n')
        for i, T in enumerate(cameras):
            pos = T[:3,3]
            quat = R_scipy.from_matrix(T[:3,:3]).as_quat()
            f.write(f"{i} {pos[0]} {pos[1]} {pos[2]} {quat[0]} {quat[1]} {quat[2]} {quat[3]}\n")
    print(f"Saved PLY: {ply}\nSaved trajectory: {traj}")


def save_observation_data(
    obs_uv: np.ndarray,
    cam_idxs: np.ndarray,
    pt_idxs: np.ndarray,
    point_observations: dict,
    output_dir: str,
    basename: str = "observations"
):
    """
    Save sparse-SfM observation data to disk:
      - obs_uv, cam_idxs, pt_idxs as a compressed .npz
      - point_observations dict as a pickle
    """
    os.makedirs(output_dir, exist_ok=True)
    npz_path = os.path.join(output_dir, f"{basename}.npz")
    pkl_path = os.path.join(output_dir, f"{basename}.pkl")
    # Save arrays
    np.savez_compressed(npz_path, obs_uv=obs_uv, cam_idxs=cam_idxs, pt_idxs=pt_idxs)
    # Save dict
    with open(pkl_path, "wb") as f:
        pickle.dump(point_observations, f)
    print(f"Saved observation arrays to: {npz_path}")
    print(f"Saved observation dict to:   {pkl_path}")


# %%



In [105]:
#save_outputs(points, cams, OUTPUT_DIR)
save_outputs(opt_pts, opt_cams, OUTPUT_DIR)
save_observation_data(
    obs_uv,
    camera_idxs,
    point_idxs,
    point_observations,
    OUTPUT_DIR
)

Saved PLY: D:\lyz\Python\CV\AI4701\final project\ablation_output\optimized_point_cloud.ply
Saved trajectory: D:\lyz\Python\CV\AI4701\final project\ablation_output\optimized_camera_trajectory.txt
Saved observation arrays to: D:\lyz\Python\CV\AI4701\final project\ablation_output\observations.npz
Saved observation dict to:   D:\lyz\Python\CV\AI4701\final project\ablation_output\observations.pkl


## Cell 9: Visualization

In [106]:

pcd = o3d.io.read_point_cloud(os.path.join(OUTPUT_DIR, 'optimized_point_cloud.ply'))
points = np.asarray(pcd.points)
poses_vis = []
with open(os.path.join(OUTPUT_DIR, 'optimized_camera_trajectory.txt'), 'r') as f:
    for line in f:
        if line.startswith('#'): continue
        parts = line.split()
        _, tx, ty, tz, qx, qy, qz, qw = parts
        Rm = R_scipy.from_quat([float(qx),float(qy),float(qz),float(qw)]).as_matrix()
        poses_vis.append({'R': Rm, 't': np.array([float(tx),float(ty),float(tz)]).reshape(3,1)})

print(f"Loaded {len(points)} points and {len(poses_vis)} cameras.")
visualize_with_open3d(points, poses_vis,show_cameras=True)

camera_mats = []
for pose in poses_vis:
    T = np.eye(4, dtype=np.float64)
    T[:3,:3] = pose['R']
    T[:3, 3] = pose['t'].reshape(3)
    camera_mats.append(T)
camera_mats = np.stack(camera_mats, axis=0)  # → (N_cam,4,4)

visualize_colored_point_cloud(points, camera_mats ,valid_paths ,K)

Loaded 1338 points and 10 cameras.


## Cell 10: Evaluation

In [107]:
def load_observation_data(
    npz_path: str,
    pkl_path: str
) -> tuple:
    """
    Load sparse-SfM observation data from disk.
    Returns: obs_uv, cam_idxs, pt_idxs, point_observations
    """
    # Load arrays
    data = np.load(npz_path)
    obs_uv = data["obs_uv"]
    cam_idxs = data["cam_idxs"]
    pt_idxs = data["pt_idxs"]
    # Load dict
    with open(pkl_path, "rb") as f:
        point_observations = pickle.load(f)
    print(f"Loaded observation arrays from: {npz_path}")
    print(f"Loaded observation dict from:   {pkl_path}")
    return obs_uv, cam_idxs, pt_idxs, point_observations

In [108]:
obs_uv2, cam_idxs2, pt_idxs2, point_observations2 = load_observation_data(
    os.path.join(OUTPUT_DIR, "observations.npz"),
    os.path.join(OUTPUT_DIR, "observations.pkl")
)

# 1) 计算重投影误差
errs = reprojection_errors(points, camera_mats, K, obs_uv2, cam_idxs2, pt_idxs2)
stats = reprojection_stats(errs)
print("Reproj error (px):", stats)

# 2) 点轨迹长度
track_stats = track_length_stats(point_observations)
print("Track lengths:", track_stats)

# 3) 图像覆盖度
cov_stats = image_coverage_stats(point_observations, camera_mats.shape[0])
print("Image coverage:", cov_stats)

# 4) 点/相机 比例
ratio = points_per_camera_ratio(points, camera_mats)
print("Points/Cameras:", ratio)


Loaded observation arrays from: D:\lyz\Python\CV\AI4701\final project\ablation_output\observations.npz
Loaded observation dict from:   D:\lyz\Python\CV\AI4701\final project\ablation_output\observations.pkl
Reproj error (px): {'mean': 13.810813964218728, 'median': 6.692110194140035, 'max': 361.3033171065232, 'std': 26.957382800252578}
Track lengths: {'mean': 2.0, 'median': 2.0, 'max': 2, 'hist': array([   0,    0, 1338], dtype=int64)}
Image coverage: {'per_image': array([  51,   34,    0, 1224,  792,    0,  503,    4,   22,   46]), 'mean': 267.6, 'median': 40.0, 'min': 0, 'max': 1224}
Points/Cameras: 133.8
